In [2]:
import pandas as pd
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup

In [3]:
navigator = 'Microsoft Edge (Scrapping; This is my system)'
url = 'https://www.auto-data.net'

In [4]:
html = requests.get(url, headers={'User-Agent': navigator})
html

<Response [200]>

In [5]:
soup = BeautifulSoup(html.text, 'html.parser')

In [126]:
brands = soup.find_all('a', {'class' : 'marki_blok'})
brands[1]

<a class="marki_blok" href="/en/alfa-romeo-brand-11" title="Alfa Romeo - Technical Specs, Fuel consumption, Dimensions"><img alt="Alfa Romeo - Technical Specs, Fuel consumption, Dimensions" src="/img/logos/Alfa_Romeo.png"/><strong>Alfa Romeo</strong></a>

In [131]:
brand_list = re.findall('href="(\S+)"',str(brands))
brand_list

['/en/acura-brand-6',
 '/en/alfa-romeo-brand-11',
 '/en/alpina-brand-16',
 '/en/aston-martin-brand-36',
 '/en/audi-brand-41',
 '/en/bentley-brand-66',
 '/en/bmw-brand-86',
 '/en/bugatti-brand-106',
 '/en/cadillac-brand-121',
 '/en/chevrolet-brand-156',
 '/en/chrysler-brand-161',
 '/en/citroen-brand-166',
 '/en/cupra-brand-256',
 '/en/dacia-brand-181',
 '/en/daewoo-brand-191',
 '/en/daihatsu-brand-2',
 '/en/dodge-brand-32',
 '/en/ds-brand-198',
 '/en/ferrari-brand-62',
 '/en/fiat-brand-67',
 '/en/ford-brand-72',
 '/en/gmc-brand-97',
 '/en/great-wall-brand-107',
 '/en/honda-brand-127',
 '/en/hummer-brand-137',
 '/en/hyundai-brand-147',
 '/en/infiniti-brand-152',
 '/en/jaguar-brand-3',
 '/en/jeep-brand-8',
 '/en/kia-brand-23',
 '/en/koenigsegg-brand-28',
 '/en/lada-brand-140',
 '/en/lamborghini-brand-38',
 '/en/lancia-brand-43',
 '/en/land-rover-brand-48',
 '/en/lexus-brand-58',
 '/en/lotus-brand-78',
 '/en/maserati-brand-108',
 '/en/mazda-brand-118',
 '/en/mclaren-brand-123',
 '/en/merce

In [22]:
for urls in brand_list:
    

In [ ]:
page2 = 'https://www.auto-data.net/fr/acura-cl-model-138'         #Inside each brand, find the models

In [ ]:
page3 = 'https://www.auto-data.net/fr/acura-cl-ii-generation-7196'    #Inside each model, find the correct generation/series

In [ ]:
page4 = 'https://www.auto-data.net/fr/acura-cl-ii-3.2-i-v6-24v-type-s-263hp-automatic-31069'   #This is the final page, retrieve the desired information from here

#